In [1]:
import os
TestImageDir = "../DATASET/SeekShot/"
Fiels = os.listdir(TestImageDir)

#TestImageDir = "../DATASET/FLIR/video/AnnotatedPreviewData/"
imageNum = 50
imageIdx = [4,5,6,9]
Modeldir = "../Model/FasterRCNN/ModelSample/"
ModelEpoch = 20
saveDir = "../Results/Video/Sample/"
ZIP = True
batchSize = 2

In [2]:
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import torch.utils.data
import torch.nn as nn
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.transforms import ToTensor
#from tqdm.notebook import tqdm
from tqdm import tqdm
import os
import json
import utils

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        self.imgs = list(sorted(os.listdir(os.path.join(root))))
        self.Kimg = []
    def __getitem__(self, idx):
        img_path = os.path.join(self.root,  self.imgs[idx])
        img = Image.open(img_path).convert("RGB")
        target = []
        
        if self.transforms is not None:
            img = self.transforms(img, target)
        return img

    def __len__(self):
        return len(self.imgs)

In [ ]:
def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [ ]:
def get_instance_segmentation_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, 
                                                                 box_nms_thresh = 0.35)
    
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    
    return model

In [ ]:
dataset_test = Dataset(TestImageDir, None)
#indices = torch.randperm(len(dataset_test)).tolist()
indices = [i for i in range(len(dataset_test))]
dataset_test = torch.utils.data.Subset(dataset_test, indices[0:-1])
data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=2, shuffle=False, num_workers=1,
    collate_fn=utils.collate_fn)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = get_instance_segmentation_model(6)
model.load_state_dict(torch.load(Modeldir + "epoch" + str(ModelEpoch) + ".pth"))

In [ ]:
model.to(device)
try:
    os.mkdir(saveDir)
except OSError:
    pass

file_list = sorted(os.listdir(TestImageDir + Fiels[0]))
file_list[:4]

In [ ]:
for j in tqdm(range(len(dataset_test))):
    img = ToTensor()(dataset_test[j])
    model.eval()
    with torch.no_grad():
        prediction = model([img.to(device)])


    plt.figure(figsize = (15,15))
    plt.imshow(img.permute(1, 2, 0))

    color_set = ['w','r','b','y','g','m']  #각 클래스별로 다른 색깔
    The_boxes = (prediction[0]['boxes']).tolist()
    this_labels = (prediction[0]['labels']).tolist()
    
    for i, box in enumerate(The_boxes):
      x_values = [box[0], box[0], box[2], box[2], box[0]]
      y_values = [box[1], box[3], box[3], box[1], box[1]]
      plt.plot(x_values, y_values, linewidth = 3, color = color_set[this_labels[i]])

    plt.savefig(saveDir + 'result_'+ file_list[j], bbox_inches="tight")
    plt.close()